In [ ]:
import numpy as np
import sys
import os
import plotly.graph_objects as go
import plotly.express as px
from scipy.ndimage import uniform_filter1d, gaussian_filter1d
import pandas as pd
import pymap3d as pm
from plotly.subplots import make_subplots
sys.path.append('../Functions/py_functions/') # This path is so that within each function file, you can import the other function files with relative paths
sys.path.append('../') # This path is so that we can import the functions folder from the root directory compared to where this file is


In [ ]:
# load the tire data file
# tire_data = pd.read_csv('../Data/logs/dataLog00121.TXT')
# tire_data = pd.read_csv('../Data/logs/Data71.csv')
data_num = "18"
dt = np.dtype([('time', '<u4'),
('micros', '<u4'),
('temp', 'f4'),
('aX', 'f4'),
('aY', 'f4'),
('aZ', 'f4'),
('gX', 'f4'),
('gY', 'f4'),
('gZ', 'f4'),
('dt', '<u4'),
('queue', '<u4'),])
with open(f'../Data/logs/Data{data_num}.bin', 'rb') as f:
    b = f.read()
# truncate to the nearest multiple of the dtype size
if len(b) % dt.itemsize != 0:
    b = b[:-(len(b) % dt.itemsize)]
data = np.frombuffer(b, dtype=dt)
data = np.sort(data, order='micros')
tire_data = pd.DataFrame(data[50:])
gpsdt = np.dtype([('latitude', 'i4'),
('longitude', 'i4'),
('altitude', 'i4'),
('fixType', 'u1'),
('carrSoln', 'u1'),
('bs', 'u2'),
('hAcc', 'i4'),
('vAcc', 'i4'),
('micros', 'u4'),
('sensor_id', '>u1'),
('bss', 'u2'),
('bsss', 'u1'),])
print(gpsdt.itemsize)
with open(f'../Data/logs/gpsData{data_num}.bin', 'rb') as f:
    b = f.read()
# truncate to the nearest multiple of the dtype size
if len(b) % gpsdt.itemsize != 0:
    b = b[:-(len(b) % gpsdt.itemsize)]
gpsdata = np.frombuffer(b, dtype=gpsdt)
gpsdata = np.sort(gpsdata, order='micros')
gps_data = pd.DataFrame(gpsdata[5:])
gps_data['latitude'] = gps_data['latitude'] * 1e-7
gps_data['longitude'] = gps_data['longitude'] * 1e-7
gps_data['altitude'] = gps_data['altitude'] * 1e-3
# separate the gps data by sensor id
# print(gps_data['sensor_id'])
for i in range(0, gps_data['sensor_id'].max()+1):
    globals()[f'gps_data_{i}'] = gps_data[gps_data['sensor_id'] == i]


In [ ]:
tire_data['aX'] = tire_data['aX'] / 9.81
tire_data['aY'] = tire_data['aY'] / 9.81
tire_data['aZ'] = tire_data['aZ'] / 9.81

# swap x and y axes
tire_data['aX'], tire_data['aY'] = tire_data['aY'], tire_data['aX'].copy()
tire_data['gX'], tire_data['gY'] = tire_data['gY'], tire_data['gX'].copy()
tire_data['raw_time'] = tire_data['time']
tire_data['time'] = tire_data['micros'] / 1000000

# make gps_data_0 the same length as gps_data_1
if len(gps_data_0) > len(gps_data_1):
    gps_data_0 = gps_data_0[:len(gps_data_1)]
elif len(gps_data_0) < len(gps_data_1):
    gps_data_1 = gps_data_1[:len(gps_data_0)]

In [ ]:
# plot the data
fig = make_subplots(rows=8, cols=1, shared_xaxes=True)
fig.add_trace(go.Scattergl(x=tire_data['time'], y=tire_data['aX'], name='aX'), row=1, col=1)
fig.add_trace(go.Scattergl(x=tire_data['time'], y=tire_data['aY'], name='aY'), row=1, col=1)
fig.add_trace(go.Scattergl(x=tire_data['time'], y=tire_data['aZ'], name='aZ'), row=1, col=1)
fig.add_trace(go.Scattergl(x=tire_data['time'], y=tire_data['gX'], name='gX'), row=2, col=1)
fig.add_trace(go.Scattergl(x=tire_data['time'], y=tire_data['gY'], name='gY'), row=2, col=1)
fig.add_trace(go.Scattergl(x=tire_data['time'], y=tire_data['gZ'], name='gZ'), row=2, col=1)
fig.add_trace(go.Scattergl(x=tire_data['time'], y=tire_data['temp'], name='Temperature'), row=3, col=1)
fig.add_trace(go.Scattergl(x=tire_data['time'], y=tire_data['queue'], name='Queue'), row=3, col=1)
if 'output_Hz' in tire_data.columns:
    fig.add_trace(go.Scattergl(x=tire_data['time'], y=tire_data['output_Hz'], name='Hz'), row=4, col=1)
else:
    fig.add_trace(go.Scattergl(x=tire_data['time'], y=1000000/tire_data['dt'], name='Hz'), row=4, col=1) # , mode="markers"
time_dt = np.zeros(len(tire_data['time']))
time_dt[1:] = np.diff(tire_data['time'])
fig.add_trace(go.Scattergl(x=tire_data['time'], y=1/time_dt, name='Hz Real'), row=4, col=1)
for i in range(0, gps_data['sensor_id'].max()+1):
    fig.add_trace(go.Scattergl(x=globals()[f'gps_data_{i}']['micros']/1000000, y=globals()[f'gps_data_{i}']['latitude'], name=f'gps_data_{i}'), row=5, col=1)
    fig.add_trace(go.Scattergl(x=globals()[f'gps_data_{i}']['micros']/1000000, y=globals()[f'gps_data_{i}']['longitude'], name=f'gps_data_{i}'), row=6, col=1)
    fig.add_trace(go.Scattergl(x=globals()[f'gps_data_{i}']['micros']/1000000, y=globals()[f'gps_data_{i}']['altitude'], name=f'gps_data_{i}'), row=7, col=1)
    fig.add_trace(go.Scattergl(x=globals()[f'gps_data_{i}']['micros']/1000000, y=globals()[f'gps_data_{i}']['hAcc'], name=f'gps_data_{i} horizontal'), row=8, col=1)
    fig.add_trace(go.Scattergl(x=globals()[f'gps_data_{i}']['micros']/1000000, y=globals()[f'gps_data_{i}']['vAcc'], name=f'gps_data_{i} vert'), row=8, col=1)

fig.update_layout(template="plotly_dark", title_text=f"Hi", height=1000, width=1000)
fig.show()


In [ ]:
px.set_mapbox_access_token(open(".mapbox_token").read())
fig = px.scatter_mapbox(lat=[37.14035], lon=[-121.6234], size_max=15, zoom=14)
for i in range(0, gps_data['sensor_id'].max()+1):
    fig.add_scattermapbox(lat=globals()[f'gps_data_{i}']['latitude'], lon=globals()[f'gps_data_{i}']['longitude'], name=f'gps_data_{i}')
fig.update_layout(template="plotly_dark", title_text=f"Hi", height=1000, width=1000, mapbox_style="mapbox://styles/eppicjr/ck4qx60vr00jy1co6u7ckgg6e")
fig.show()

In [ ]:
# Convert to enu from lat/lon
datum = [37.14035, -121.6234]
fig = go.Figure()
for i in range(0, gps_data['sensor_id'].max()+1):
    east, north, up = pm.geodetic2enu(globals()[f'gps_data_{i}']['latitude'], globals()[f'gps_data_{i}']['longitude'], globals()[f'gps_data_{i}']['altitude'], datum[0], datum[1], 0)
    fig.add_trace(go.Scattergl(x=east, y=north, mode='markers', marker=dict(color=globals()[f'gps_data_{i}']['micros']/1000000, colorscale='Viridis', showscale=True), name=f'gps_data_{i}'))
    globals()[f'gps_data_{i}']['east'] = east
    globals()[f'gps_data_{i}']['north'] = north
    globals()[f'gps_data_{i}']['up'] = up
    # calculate heading and speed
    globals()[f'gps_data_{i}']['heading'] = np.zeros(len(globals()[f'gps_data_{i}']['micros']))
    globals()[f'gps_data_{i}']['speed'] = np.zeros(len(globals()[f'gps_data_{i}']['micros']))
    # vel in enu
    globals()[f'gps_data_{i}']['vel_east'] = np.zeros(len(globals()[f'gps_data_{i}']['micros']))
    globals()[f'gps_data_{i}']['vel_north'] = np.zeros(len(globals()[f'gps_data_{i}']['micros']))
    globals()[f'gps_data_{i}']['vel_up'] = np.zeros(len(globals()[f'gps_data_{i}']['micros']))
    globals()[f'gps_data_{i}']['vel_east'][1:] = np.diff(globals()[f'gps_data_{i}']['east'])/np.diff(globals()[f'gps_data_{i}']['micros'])*1000000
    globals()[f'gps_data_{i}']['vel_north'][1:] = np.diff(globals()[f'gps_data_{i}']['north'])/np.diff(globals()[f'gps_data_{i}']['micros'])*1000000
    globals()[f'gps_data_{i}']['vel_up'][1:] = np.diff(globals()[f'gps_data_{i}']['up'])/np.diff(globals()[f'gps_data_{i}']['micros'])*1000000
    # heading and speed
    globals()[f'gps_data_{i}']['heading'][1:] = np.arctan2(globals()[f'gps_data_{i}']['vel_east'][1:], globals()[f'gps_data_{i}']['vel_north'][1:])
    globals()[f'gps_data_{i}']['speed'][1:] = np.sqrt(globals()[f'gps_data_{i}']['vel_east'][1:]**2 + globals()[f'gps_data_{i}']['vel_north'][1:]**2)

fig.update_layout(template="plotly_dark", title_text=f"Hi", height=1000, width=1000)
fig.show()
    

In [ ]:
# Distance between points
dist = np.sqrt((np.array(gps_data_0['east']) - np.array(gps_data_1['east']))**2 + (np.array(gps_data_0['north']) - np.array(gps_data_1['north']))**2)
angle = np.arctan2(np.array(gps_data_0['east']) - np.array(gps_data_1['east']), np.array(gps_data_0['north']) - np.array(gps_data_1['north']))
print(dist)
fig = make_subplots(rows=3, cols=1, shared_xaxes=True)
fig.add_trace(go.Scattergl(x=gps_data_0['micros']/1000000, y=dist, name='dist'), row=1, col=1)
fig.add_trace(go.Scattergl(x=gps_data_0['micros']/1000000, y=np.rad2deg(angle), name='angle'), row=2, col=1)
for i in range(0, gps_data['sensor_id'].max()+1):
    fig.add_trace(go.Scattergl(x=globals()[f'gps_data_{i}']['micros']/1000000, y=globals()[f'gps_data_{i}']['speed'], name=f'gps_data_{i}'), row=3, col=1)
    fig.add_trace(go.Scattergl(x=globals()[f'gps_data_{i}']['micros']/1000000, y=np.rad2deg(globals()[f'gps_data_{i}']['heading']), name=f'gps_data_{i}'), row=2, col=1)
fig.update_layout(template="plotly_dark", title_text=f"Hi", height=1000, width=1000)
fig.show()